In [1]:
import numpy as np
import cv2
import os
import json
import utils

In [2]:
#input_directory = r'images\processed\non_cropped\\'
#target_directory = r'output\session_1\grabCut\\'

input_directory = r'images\processed\session_3\\'
target_directory = r'output\session_3\grabcut\\'

In [3]:
images_name = []

for name in os.listdir(input_directory):
    if name.endswith(".jpg"):
        images_name.append(name)

In [4]:
images = {}

for name in images_name:
    images[name] = cv2.imread(input_directory + name)

In [5]:
bboxes = json.loads(open(input_directory + "images_bboxes.json", "r").read())

### Bbox

In [6]:
for name in images_name:
    image = images[name]
    bbox = bboxes[name]
    for i in range(len(bbox)):
        cv2.imwrite(target_directory + f'BBOX_{name[:-4]}_{i+1}.jpg', utils.drawBbox(image, bbox[i]))

### Mask and ouput

In [7]:
def applyGrabCutAndWriteOutputs(image, bbox, name, num):
    if num > 0:
        _, outputMask = utils.grabCut(image, bbox)
        output = cv2.bitwise_and(image, image, mask=outputMask)
        cv2.imwrite(target_directory + f'DEFBGD_MASK_{name}_{num}.jpg', outputMask)
        cv2.imwrite(target_directory + f'DEFBGD_OUTPUT_{name}_{num}.jpg', output)

        _, outputMask = utils.grabCut(image, utils.bboxToMask(image, bbox, cv2.GC_PR_BGD), mode = 'MASK')
        output = cv2.bitwise_and(image, image, mask=outputMask)
        cv2.imwrite(target_directory + f'PROBBGD_MASK_{name}_{num}.jpg', outputMask)
        cv2.imwrite(target_directory + f'PROBBGD_OUTPUT_{name}_{num}.jpg', output)
    else:
        _, outputMask = utils.grabCut(image, bbox)
        output = cv2.bitwise_and(image, image, mask=outputMask)
        cv2.imwrite(target_directory + f'FULLBBOX_MASK_{name}.jpg', outputMask)
        cv2.imwrite(target_directory + f'FULLBBOX_OUTPUT_{name}.jpg', output)

In [8]:
for name in images_name:
    image = images[name]
    applyGrabCutAndWriteOutputs(image, utils.fullBoundingBox(image), name[:-4], 0)

In [9]:
for name in images_name:
    image = images[name]
    bbox = bboxes[name]
    for i in range(len(bbox)):
        applyGrabCutAndWriteOutputs(image, bbox[i], name[:-4], i+1)